In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

%matplotlib inline
from IPython.core.pylabtools import figsize

In [2]:
tf.__version__

'1.8.0'

In [12]:
tf.reset_default_graph()

need to be able to pad sequences to be the same length

In [13]:
# shape = (batch size, max length of sentence in batch)
#word_ids = tf.placeholder(tf.int32, shape=[None, None])

# start joe
max_len = 11
word_ids = tf.placeholder(tf.int32, shape=[None, max_len])
# end joe

# shape = (batch size)
sequence_lengths = tf.placeholder(tf.int32, shape=[None])

assume "embeddings" is a 2d numpy array, so embeddings[i] is the embedding of the ith word

In [14]:
# start my stuff
num_words = 100 
embed_dim = 50  
embeddings = np.zeros((num_words, embed_dim))
# end my stuff

L = tf.Variable(embeddings, dtype=tf.float32, trainable=False)
# shape = (batch, sentence, word_vector_size)
pretrained_embeddings = tf.nn.embedding_lookup(L, word_ids)

also need to be able to pad word representations to same length

In [15]:
# shape = (batch size, max length of sentence, max length of word)
#char_ids = tf.placeholder(tf.int32, shape=[None, None, None])

# joe start
#batch_size = 10
max_sent_len = 17
max_word_len = 23
char_ids = tf.placeholder(tf.int32, shape=[None, max_sent_len, max_word_len])
# joe end

# shape = (batch_size, max_length of sentence)
#word_lengths = tf.placeholder(tf.int32, shape=[None, None])

# joe start
#max_word_len = 13
word_lengths = tf.placeholder(tf.int32, shape=[None, max_word_len])
# joe end

In [16]:
char_ids

<tf.Tensor 'Placeholder_2:0' shape=(?, 17, 23) dtype=int32>

In [17]:
char_embeddings

<tf.Tensor 'Reshape:0' shape=(?, ?, ?) dtype=float32>

In [18]:
# start joe
nchars = 10
dim_char = 10
word_lengths = np.zeros(10)
char_hidden_size = 10
# end joe

# 1. get character embeddings
K = tf.get_variable(name="char_embeddings", dtype=tf.float32,
    shape=[nchars, dim_char])
# shape = (batch, sentence, word, dim of char embeddings)
#char_embeddings = tf.nn.embedding_lookup(K, char_ids)

# joe start
char_embeddings = tf.nn.embedding_lookup(K, char_ids)
char_embeddings.set_shape([None, max_sent_len, max_word_len, dim_char])
# joe end

# 2. put the time dimension on axis=1 for dynamic_rnn
s = tf.shape(char_embeddings) # store old shape
3# shape = (batch x sentence, word, dim of char embeddings)
#char_embeddings = tf.reshape(char_embeddings, shape=[-1, s[-2], s[-1]])
word_lengths = tf.reshape(word_lengths, shape=[-1])

# 3. bi lstm on chars
cell_fw = tf.contrib.rnn.LSTMCell(char_hidden_size, state_is_tuple=True)
cell_bw = tf.contrib.rnn.LSTMCell(char_hidden_size, state_is_tuple=True)

_, ((_, output_fw), (_, output_bw)) = tf.nn.bidirectional_dynamic_rnn(cell_fw,
    cell_bw, char_embeddings, sequence_length=word_lengths,
    dtype=tf.float32)
# shape = (batch x sentence, 2 x char_hidden_size)
output = tf.concat([output_fw, output_bw], axis=-1)

# shape = (batch, sentence, 2 x char_hidden_size)
char_rep = tf.reshape(output, shape=[-1, s[1], 2*char_hidden_size])

# shape = (batch, sentence, 2 x char_hidden_size + word_vector_size)
word_embeddings = tf.concat([pretrained_embeddings, char_rep], axis=-1)

ValueError: Input 0 of layer lstm_cell_1 is incompatible with the layer: expected ndim=2, found ndim=3. Full shape received: [None, 23, 10]

In [19]:
char_embeddings

<tf.Tensor 'embedding_lookup_1:0' shape=(?, 17, 23, 10) dtype=float32>

In [38]:
tf.nn.bidirectional_dynamic_rnn?

Signature: tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, inputs, sequence_length=None, initial_state_fw=None, initial_state_bw=None, dtype=None, parallel_iterations=None, swap_memory=False, time_major=False, scope=None)
Docstring:
Creates a dynamic version of bidirectional recurrent neural network.

Takes input and builds independent forward and backward RNNs. The input_size
of forward and backward cell must match. The initial state for both directions
is zero by default (but can be set optionally) and no intermediate states are
ever returned -- the network is fully unrolled for the given (passed in)
length(s) of the sequence(s) or completely unrolled if length(s) is not
given.

Args:
  cell_fw: An instance of RNNCell, to be used for forward direction.
  cell_bw: An instance of RNNCell, to be used for backward direction.
  inputs: The RNN inputs.
    If time_major == False (default), this must be a tensor of shape:
      `[batch_size, max_time, ...]`, or a nested tuple of such elem

In [20]:
import tensorflow.contrib

In [21]:
tf.contrib.crf.crf_log_likelihood?

Signature: tf.contrib.crf.crf_log_likelihood(inputs, tag_indices, sequence_lengths, transition_params=None)
Docstring:
Computes the log-likelihood of tag sequences in a CRF.

Args:
  inputs: A [batch_size, max_seq_len, num_tags] tensor of unary potentials
      to use as input to the CRF layer.
  tag_indices: A [batch_size, max_seq_len] matrix of tag indices for which we
      compute the log-likelihood.
  sequence_lengths: A [batch_size] vector of true sequence lengths.
  transition_params: A [num_tags, num_tags] transition matrix, if available.
Returns:
  log_likelihood: A [batch_size] `Tensor` containing the log-likelihood of
    each example, given the sequence of tag indices.
  transition_params: A [num_tags, num_tags] transition matrix. This is either
      provided by the caller or created in this function.
File:      /anaconda3/lib/python3.6/site-packages/tensorflow/contrib/crf/python/ops/crf.py
Type:      function


In [22]:
tf.contrib.layers.embed_sequence?

Signature: tf.contrib.layers.embed_sequence(ids, vocab_size=None, embed_dim=None, unique=False, initializer=None, regularizer=None, trainable=True, scope=None, reuse=None)
Docstring:
Maps a sequence of symbols to a sequence of embeddings.

Typical use case would be reusing embeddings between an encoder and decoder.

Args:
  ids: `[batch_size, doc_length]` `Tensor` of type `int32` or `int64`
    with symbol ids.
  vocab_size: Integer number of symbols in vocabulary.
  embed_dim: Integer number of dimensions for embedding matrix.
  unique: If `True`, will first compute the unique set of indices, and then
       lookup each embedding once, repeating them in the output as needed.
  initializer: An initializer for the embeddings, if `None` default for
      current scope is used.
  regularizer: Optional regularizer for the embeddings.
  trainable: If `True` also add variables to the graph collection
    `GraphKeys.TRAINABLE_VARIABLES` (see `tf.Variable`).
  scope: Optional string specifying

In [23]:
tf.contrib.estimator.poisson_regression_head?

Signature: tf.contrib.estimator.poisson_regression_head(weight_column=None, label_dimension=1, loss_reduction='weighted_sum_over_batch_size', compute_full_loss=True, name=None)
Docstring:
Creates a `_Head` for poisson regression using `tf.nn.log_poisson_loss`.

The loss is the weighted sum over all input dimensions. Namely, if the input
labels have shape `[batch_size, label_dimension]`, the loss is the weighted
sum over both `batch_size` and `label_dimension`.

The head expects `logits` with shape `[D0, D1, ... DN, label_dimension]`.
In many applications, the shape is `[batch_size, label_dimension]`.

The `labels` shape must match `logits`, namely
`[D0, D1, ... DN, label_dimension]`. If `label_dimension=1`, shape
`[D0, D1, ... DN]` is also supported.

If `weight_column` is specified, weights must be of shape
`[D0, D1, ... DN]`, `[D0, D1, ... DN, 1]` or
`[D0, D1, ... DN, label_dimension]`.

This is implemented as a generalized linear model, see
https://en.wikipedia.org/wiki/Generalized_

In [24]:
import multiprocessing as mp

In [25]:
mp.Pool?

Signature: mp.Pool(processes=None, initializer=None, initargs=(), maxtasksperchild=None)
Docstring: Returns a process pool object
File:      /anaconda3/lib/python3.6/multiprocessing/context.py
Type:      method


In [26]:
mp.cpu_count()

4